In [1]:
import tensorflow as tf
import os

In [2]:
train_dataset = "cropped_train_dataset"
test_dataset = "cropped_test_dataset"

image_size = (224, 224)

batch_size = 256
buffer_size = batch_size * 2

auto = tf.data.AUTOTUNE

LEARNING_RATE = 0.0001
STEPS_PER_EPOCH = 50
VALIDATION_STEPS = 10
EPOCHS = 10

output_path = "output"
model_path = os.path.join(output_path, "siamese_network")
output_image_path = os.path.join(output_path, "output_image.png")

In [3]:
import tensorflow as tf
import numpy as np
import random
import os

In [4]:
class MapFunction():
  def __init__(self, imageSize):
    self.imageSize = imageSize

  def decode_and_resize(self, imagePath):
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)

    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, self.imageSize)

    return image

  def __call__(self, anchor, positive, negative):
    anchor = self.decode_and_resize(anchor)
    positive = self.decode_and_resize(positive)
    negative = self.decode_and_resize(negative)
    return (anchor, positive, negative)

In [5]:
class TripletGenerator:
  def __init__(self, datasetPath):
    self.peopleNames=list()

    for folderName in os.listdir(datasetPath):
      absoluteFolderName = os.path.join(datasetPath, folderName)

      numImages = len(os.listdir(absoluteFolderName))

      if numImages > 1:
        self.peopleNames.append(absoluteFolderName)

      self.allPeople = generate_all_people_dict()

  def generate_all_people_dict(self):
    allPeople = dict()

    for personName in self.peopleNames:
      imageNames = os.listdir(personName)

      personPhotos = [
          os.path.join(personName, imageName) for imageName in imageNames
      ]
      allPeople[personName] = personPhotos

    return allPeople

  def get_next_element(self):
    while True:
      anchorName = random.choice(self.peopleNames)
      temporaryNames =self.peopleNames.copy()
      temporaryNames.remove(anchorName)
      negativeName = random.choice(temporaryNames)
      # draw two images from the anchor folder without replacement
      (anchorPhoto, positivePhoto) = np.random.choice(
				a=self.allPeople[anchorName],
				size=2,
				replace=False
			)
			# draw an image from the negative folder
      negativePhoto = random.choice(self.allPeople[negativeName])
			# yield the anchor, positive and negative photos
      yield (anchorPhoto, positivePhoto, negativePhoto)

In [6]:
from imutils.paths import list_images
from tqdm import tqdm
import numpy as np
import argparse
import cv2
import os

In [8]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/siamese-201/siamese-201.zip
!unzip -qq siamese-201.zip
%cd siamese-201

--2023-08-14 02:26:09--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/siamese-201/siamese-201.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.92.177.2, 52.92.144.162, 52.218.177.89, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.92.177.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11107236 (11M) [binary/octet-stream]
Saving to: ‘siamese-201.zip’

siamese-201.zip     100%[===================>]  10.59M  20.4MB/s    in 0.5s    

2023-08-14 02:26:09 (20.4 MB/s) - ‘siamese-201.zip’ saved [11107236/11107236]

/content/siamese-201


In [9]:
! wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz -O lfw.tgz
! tar -zxf lfw.tgz
! mv "lfw-deepfunneled" "train_dataset"

--2023-08-14 02:26:47--  http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108761145 (104M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 103.72M  1.55MB/s    in 88s     

2023-08-14 02:28:16 (1.17 MB/s) - ‘lfw.tgz’ saved [108761145/108761145]



In [10]:
train_args = {
	"dataset": "train_dataset",
	"output": "cropped_train_dataset",
	"prototxt": "face_crop_model/deploy.prototxt.txt",
	"model": "face_crop_model/res10_300x300_ssd_iter_140000.caffemodel",
	"confidence": 0.5,
}

test_args = {
	"dataset": "test_dataset",
	"output": "cropped_test_dataset",
	"prototxt": "face_crop_model/deploy.prototxt.txt",
	"model": "face_crop_model/res10_300x300_ssd_iter_140000.caffemodel",
	"confidence": 0.5,
}

In [12]:
args = train_args

In [13]:
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

# check if the output dataset directory exists, if it doesn't, then
# create it
if not os.path.exists(args["output"]):
	os.makedirs(args["output"])

# grab the file and sub-directory names in dataset directory
print("[INFO] grabbing the names of files and directories...")
names = os.listdir(args["dataset"])

[INFO] loading model...
[INFO] grabbing the names of files and directories...


In [14]:
# loop over all names
print("[INFO] starting to crop faces and saving them to disk...")
for name in tqdm(names):
	# build directory path
	dirPath = os.path.join(args["dataset"], name)

	# check if the directory path is a directory
	if os.path.isdir(dirPath):
		# grab the path to all the images in the directory
		imagePaths = list(list_images(dirPath))

		# build the path to the output directory
		outputDir = os.path.join(args["output"], name)

		# check if the output directory exists, if it doesn't, then
		# create it
		if not os.path.exists(outputDir):
			os.makedirs(outputDir)

		# loop over all image paths
		for imagePath in imagePaths:
			# grab the image ID, load the image, and grab the
			# dimensions of the image
			imageID = imagePath.split(os.path.sep)[-1]
			image = cv2.imread(imagePath)
			(h, w) = image.shape[:2]

			# construct an input blob for the image by resizing to a
			# fixed 300x300 pixels and then normalizing it
			blob = cv2.dnn.blobFromImage(cv2.resize(image,
				(300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

			# pass the blob through the network and obtain the
			# detections and predictions
			net.setInput(blob)
			detections = net.forward()

			# extract the index of the detection with max
			# probability and get the maximum confidence value
			i = np.argmax(detections[0, 0, :, 2])
			confidence = detections[0, 0, i, 2]

			# filter out weak detections by ensuring the
			# `confidence` is greater than the minimum confidence
			if confidence > args["confidence"]:
				# grab the maximum dimension value
				maxDim = np.max(detections[0, 0, i, 3:7])

				# check if max dimension value is greater than one,
				# if so, skip the detection since it is erroneous
				if maxDim > 1.0:
					continue

				# clip the dimension values to be between 0 and 1
				box = np.clip(detections[0, 0, i, 3:7], 0.0, 1.0)

				# compute the (x, y)-coordinates of the bounding
				# box for the object
				box = box * np.array([w, h, w, h])
				(startX, startY, endX, endY) = box.astype("int")

				# grab the face from the image, build the path to
				# the output face image, and write it to disk
				face = image[startY:endY,startX:endX,:]
				facePath = os.path.join(outputDir, imageID)
				cv2.imwrite(facePath, face)

print("[INFO] finished cropping faces and saving them to disk...")

[INFO] starting to crop faces and saving them to disk...


100%|██████████| 5749/5749 [11:41<00:00,  8.20it/s]

[INFO] finished cropping faces and saving them to disk...
